1. Run BERT with customized embeddings
2. remove more words as stopwords?
3. why do the same words show up in multiple topics?
4. disaggregate by type and run an overall model

run once and reduce to distinct number of clusters
remove hashtags
keep users but remove @
remove stop words after model is run BERTopic().get_topic_info()
bertweet for customized embeddings 

#embeddings = pd.read_csv("hate_counter_embeddings.csv") change to array
HAVE TO REMOVE COVID

change minimum topic size
change parameters on umap and hdbscan
cluster_selection_epsilon


For BERT, we first removed all Twitter- and web-specific content such as URLs, usernames, hashtags, and emojis, and then used the hidden layer representation of the tweet to generate 768- dimensional tweet BERT-embeddings.

In [67]:
import tweepy as tw
import numpy as np
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
import gensim
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maggiewu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/maggiewu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
file_path = "hydrated.csv"
colnames = ['tweet_id', 'text', 'num_likes', 'date', 'author']
hydrated_data = pd.read_csv(file_path, names = colnames)
hydrated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72388 entries, 0 to 72387
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   72388 non-null  int64 
 1   text       72388 non-null  object
 2   num_likes  72388 non-null  int64 
 3   date       72388 non-null  object
 4   author     72388 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.8+ MB


In [69]:
og_data = pd.read_csv("all_tweet_ids.csv")

labels = (og_data.loc[(og_data.Label == 'Hate') | (og_data.Label == 'Counterhate')]\
                     .rename(columns = {"Tweet ID" : "tweet_id"})
             )

In [70]:
#join label information to hydrated data
hydrated_data1 = pd.merge(hydrated_data, labels, how = 'left', on = 'tweet_id')

In [71]:
hydrated_data1.head()
#hydrated_data1.info()

,tweet_id,text,num_likes,date,author,Label
0,1225778870093262849,@realDonaldTrump China has been trying to kill...,0,2020-02-07 13:50:31,GailRusso19,Hate
1,1242461158654619649,"Ignored warnings, shielded China: How WHO coll...",0,2020-03-24 14:39:58,apivatkumara,Hate
2,1239916262479638528,@RyanAFournier @realDonaldTrump It’s not the C...,4,2020-03-17 14:07:28,cameronssmith__,Counterhate
3,1246361730873085953,@Grammarly I though REAL intellectuals develop...,0,2020-04-04 08:59:27,Uyghur_Oghly,Hate
4,1241151165305364480,Since Wuhan virus is offensive perhaps we shou...,0,2020-03-20 23:54:31,BBoveington,Hate


Preview Embeddings

In [72]:
embeddings = pd.read_csv("hate_counter_embeddings.csv", header = None)

In [73]:
embeddings.head()
#embeddings.info()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,1244776561489911812,0.449286,-0.080693,0.423826,0.132395,0.077021,-0.246653,0.012050,0.407110,-0.088420,...,0.337652,0.068206,-0.066296,-0.134786,-0.015966,-0.165570,-0.029202,0.056983,0.250433,-0.214088
1,1244776599058370562,0.462327,0.114677,0.119433,0.090756,0.204585,-0.195269,0.154502,0.631958,0.062113,...,-0.255628,-0.091462,0.097092,-0.135727,0.127662,0.186459,0.142710,-0.131188,0.245637,0.116715
2,1244776599574085632,0.035152,-0.028830,-0.133043,-0.010387,-0.018519,-0.303503,0.023665,0.558168,0.080153,...,0.024747,0.066145,-0.242748,-0.175117,-0.092793,-0.090895,0.275401,-0.181766,-0.041805,0.180552
3,1244776603974012929,0.102819,0.295894,0.433053,0.064111,-0.141456,0.156949,0.292134,0.342282,0.147171,...,-0.008658,-0.111251,0.046656,-0.186562,-0.155140,-0.184540,0.223476,-0.085286,0.082277,0.179396
4,1244776612383674369,0.050693,-0.009848,0.161498,0.114732,0.149062,-0.283866,-0.041372,0.317175,-0.157875,...,0.054488,0.023420,-0.128858,-0.177928,0.001263,0.034888,-0.207965,-0.065526,0.155339,-0.027436


In [74]:
#ensure that the dimensions are the same
ids = hydrated_data1['tweet_id'].tolist()
small_embeddings = embeddings[embeddings[0].isin(ids)]

id2 = small_embeddings[0].tolist()
hydrated_data2 = hydrated_data1[hydrated_data1['tweet_id'].isin(id2)]
hydrated_data2 = hydrated_data2.set_index('tweet_id')
hydrated_data2 = hydrated_data2.loc[id2]

small_embeddings = small_embeddings.iloc[: , 1:]
embedding_array = small_embeddings.to_numpy()

In [75]:
#embedding_array

### Topic Modeling

In [76]:
from bertopic import BERTopic

In [77]:
# def cleaner(x):
#     import re
#     x = re.sub(r'https?:\/\/.*[\r\n]*', '', x)
#     x = re.sub("@[A-Za-z0-9]+",'', x)

#     import emoji
#     x_clean = emoji.get_emoji_regexp().sub("", x)

#     from gensim.parsing.preprocessing import remove_stopwords
#     x_final = remove_stopwords(x_clean)

#     #tokenize
#     tw_tokenizer = TweetTokenizer()
#     tokens = tw_tokenizer.tokenize(x_final)

#     #remove punctuation
#     import string
#     tokens1 = list(filter(lambda token: token not in string.punctuation, tokens))

#     #stemmer
#     from nltk.stem import SnowballStemmer
#     # Instantiate stemming class
#     snow = SnowballStemmer("english")

#     snowball_stemmed = [] 

#     for word in tokens1:
#       stem_word = snow.stem(word)  # stemming word
#       snowball_stemmed.append(stem_word)  # append to the list
    
#     #snowball_stemmed
#     cleaned = ' '.join([i+' ' for i in snowball_stemmed])
#     #data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
#     return cleaned

# clean_bertcorpus = hydrated_data1.apply(lambda x: cleaner(x['text']), axis=1)

In [78]:
def cleaner2(x):
    import re
    x = re.sub(r'https?:\/\/.*[\r\n]*', '', x)
    x = re.sub("@[A-Za-z0-9]+",'', x)
    x = x.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    
    #from gensim.parsing.preprocessing import remove_stopwords
    #x = remove_stopwords(x)
    
    import emoji
    cleaned = emoji.get_emoji_regexp().sub("", x)

    return cleaned

clean_bertcorpus = hydrated_data2.apply(lambda x: cleaner2(x['text']), axis=1)

## Using the leaf clustering algorithm

We see a lot of small homogeneous topics but none of them are particularly useful.

In [95]:
random_state = 12000

from sklearn.feature_extraction import text 

additional_stop_words = ["ur", '21', 'amb', 'blah', 'till', 'yep', 'rt', 'yall', '23', '19', 'guy', 'hes', '14',
                        'okay', 'whats', 'huh', 'jan', 'did', 'didnt', 'mean', 'does', 'youre', 'yeah', 'hey', 'just',
                        'like', 'yes', 'yessss', 'yesssss', 'plz', '40', 'agree', 'zy', 'zvinopfuura', 'yess',
                         'zzzzzzzz', 'zzzzzzzz', 'yessssss', 'yesss', 'gt', 'gtgtgt', 'etcwith', 'etchowever', 'etcwhy',
                        'etcthere', 'etcthey', 'etcpoc', 'zzzzzz', 'etcit', 'etcthe', 'etcon', 'etcl',
                        'fuck', 'etcetera', 'etcetc']
s_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer_custom = CountVectorizer(max_df = 0.70, min_df = 0.05, stop_words = s_words)

from hdbscan import HDBSCAN

#HDBSCAN
hdbscan_custom = HDBSCAN(min_cluster_size=15, metric='euclidean', 
                        cluster_selection_method='leaf', prediction_data = True, cluster_selection_epsilon = 1.0)

In [96]:
random_state = 12000
alberto = BERTopic(language='english', 
                   calculate_probabilities=True, 
                   verbose=True, 
                   vectorizer_model=vectorizer_custom,
                   hdbscan_model= hdbscan_custom
                  )

In [81]:
topics, probabilities = alberto.fit_transform(clean_bertcorpus, embedding_array)

2021-07-29 15:42:16,818 - BERTopic - Reduced dimensionality with UMAP
2021-07-29 15:48:46,116 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [82]:
alberto.save('bertopic_model')
#alberto = BERTopic.load("bertopic_model")

/Users/maggiewu/opt/anaconda3/envs/REU_2021/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [83]:
freq = alberto.get_topic_info()
freq

,Topic,Count,Name
0,-1,68318,-1_wuhanvirus_chinaliedpeopledied_stop_communist
1,0,208,0_book_tong_ting_tedros
2,1,49,1_zoonotic_france_forward_ford
3,2,47,2_vets_necessarily_cat_warn
4,3,44,3_everybody_fighting_kung_flu
5,4,40,4_shameful_ridicule_dislike_liers
6,5,38,5_hedge_tencent_terrifying_leak
7,6,37,6_fuck_designer_bitlord_ir
8,7,35,7_preach_fuck_obama_watch
9,8,34,8_fuck_zoonotic_fight_forward


In [84]:
alberto.visualize_topics()
#collapse to 5 clusters

In [85]:
alberto.visualize_barchart(top_n_topics=6, n_words=10)

In [86]:
# nltk.download('stopwords')
# from nltk.corpus import stopwords

# #appending some extra goodies to the stop word lexicon
# stopwords_list = stopwords.words('english')
# extra_stopwords = ['like', '19', 'covid19', 'virus']
# stopwords_list.extend(extra_stopwords)

# topics = all_topic_model.get_topics()
# new_topics = {}
# for i in (range(len(topics)-1)):
#     words = dict(all_topic_model.get_topic(topic=i))
#     filtered_topic_dict = {}
#     for key, value in words.items():
#         if (key not in stopwords_list):
#             filtered_topic_dict[key] = value
#     new_topics[i] = list(filtered_topic_dict.items())
#     i=i+1

# new_topics

## Using the eom clustering algorithm

Clustering parameters: Clusters must have 25 members or more. Clusters must be more than 1.0 Euclidean units away from each other.

Vectorizer parameters: Words must appear between (0.05, 0.70) of the documents in the corpus (or else they will be filtered out as stop words).

What is "nnevvy" of topic 0? cnn.com/2020/04/14/asia/nnevvy-china-taiwan-twitter-intl-hnk/index.html

In [97]:
#HDBSCAN
random_state = 12000
hdbscan_custom2 = HDBSCAN(metric='euclidean', min_cluster_size = 25, min_samples=5,
                        cluster_selection_method='eom', prediction_data = True, cluster_selection_epsilon = 1.0)
alberto2 = BERTopic(language='english', 
                    calculate_probabilities=True, 
                    verbose=True, 
                    hdbscan_model = hdbscan_custom2,
                    vectorizer_model=vectorizer_custom)
topics, probabilities = alberto2.fit_transform(clean_bertcorpus, embedding_array)

2021-07-29 16:29:42,313 - BERTopic - Reduced dimensionality with UMAP
2021-07-29 16:33:53,773 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [98]:
alberto2.get_topic_info()

,Topic,Count,Name
0,-1,68097,-1_ccp_dont_wuhan_covid19
1,0,436,0_responsible_curfew_produce_ccpvirus
2,1,102,1_petition_sign_leather_dogs
3,2,66,2_oneworld_urge_pandemic_defeat
4,3,64,3_abusing_kai_treatment_positive
5,4,54,4_die_gonna_going_arent
6,5,54,5_shameful_oneworld_urge_pandemic
7,6,47,6_serpent_god_displeasing_sunshine
8,7,46,7_cripples_hours_needed_vaccine
9,8,46,8_tencent_terrifying_leak_statistics


In [99]:
alberto2.visualize_topics()

In [100]:
alberto2.visualize_barchart(n_words=12)

# CounterHate Tweets Only

Here, we inputed custom parameters for HDBSCAN and the count vectorizer. 

Clustering parameters: Clusters must have 20 members or more. Clusters must be more than 1.0 Euclidean units away from each other.

Vectorizer parameters: Words must appear between (0.05, 0.70) of the documents in the corpus (or else they will be filtered out as stop words).

In [40]:
counterhate_tweets = hydrated_data1.loc[hydrated_data1.Label == 'Counterhate']

#ensure that the dimensions are the same
ids = counterhate_tweets['tweet_id'].tolist()
counterhate_embeddings = embeddings[embeddings[0].isin(ids)]

id2 = counterhate_embeddings[0].tolist()
counterhate_tweets2 = counterhate_tweets[counterhate_tweets['tweet_id'].isin(id2)]
counterhate_tweets2 = counterhate_tweets2.set_index('tweet_id')
counterhate_tweets2 = counterhate_tweets2.loc[id2]

counterhate_embeddings = counterhate_embeddings.iloc[: , 1:]
counterhate_embedding_array = counterhate_embeddings.to_numpy()

In [41]:
random_state = 12000

hdbscan_custom3 = HDBSCAN(metric='euclidean', cluster_selection_method='eom', prediction_data = True, 
                           cluster_selection_epsilon = 1.0, min_cluster_size = 20)

from sklearn.feature_extraction import text 

additional_stop_words = ["ur", '21', 'amb', 'blah', 'till', 'yep', 'rt', 'yall', '23', '19', 'guy', 'hes', '14',
                        'okay', 'whats', 'huh', 'jan', 'did', 'didnt', 'mean', 'does', 'youre', 'yeah', 'hey', 'just',
                        'like', 'yes', 'yessss', 'yesssss', 'plz', '40', 'agree', 'zy', 'zvinopfuura', 'yess',
                         'zzzzzzzz', 'zzzzzzzz', 'yessssss', 'yesss', 'gt', 'gtgtgt', 'etcwith', 'etchowever', 'etcwhy',
                        'etcthere', 'etcthey', 'etcpoc', 'zzzzzz', 'etcit', 'etcthe', 'etcon', 'etcl', 'pls', 'got',
                        'aint', 'arent', 'cuz', 'bc', 'aka', 'says', 'zvese', 'zvenyu', 'exp', 'zvefeja', 'exodus3414',
                        'zuuuuune', 'zuniga', 'zulum']

s_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer_custom = CountVectorizer(max_df = 0.70, min_df = 0.05, stop_words = s_words)


counterhate_corpus = counterhate_tweets2.apply(lambda x: cleaner2(x['text']), axis=1)

counterberto = BERTopic(language='english', 
                        calculate_probabilities=True, 
                        verbose=True, 
                        hdbscan_model = hdbscan_custom3,
                        vectorizer_model = vectorizer_custom)

topics, probabilities = counterberto.fit_transform(counterhate_corpus, counterhate_embedding_array)

2021-07-29 00:31:20,818 - BERTopic - Reduced dimensionality with UMAP
2021-07-29 00:31:22,076 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [42]:
counterberto.get_topic_info()

,Topic,Count,Name
0,-1,14367,-1_virus_chinese_people_covid19
1,0,66,0_preach_hope_pharma_false
2,1,41,1_shameful_ridicule_dislike_accusers
3,2,35,2_curfew_pandemic_janta_oneworld
4,3,32,3_support_usps_copy_signing
5,4,28,4_zucker_exit_exercising_exhausted
6,5,26,5_zucker_exit_exercising_exhausted
7,6,21,6_zucker_exit_exercising_exhausted


In [43]:
counterberto.visualize_topics()

In [44]:
counterberto.visualize_barchart(top_n_topics=6, n_words = 10)

These topics could be better. Below, we are going to use the HDBSCAN defaults because we received better results before changing the parameters.

In [91]:
random_state = 12000

additional_stop_words = ["ur", '21', 'amb', 'blah', 'till', 'yep', 'rt', 'yall', '23', '19', 'guy', 'hes', '14',
                        'okay', 'whats', 'huh', 'jan', 'did', 'didnt', 'mean', 'does', 'youre', 'yeah', 'hey', 'just',
                        'like', 'yes', 'yessss', 'yesssss', 'plz', '40', 'agree', 'zy', 'zvinopfuura', 'yess',
                         'zzzzzzzz', 'zzzzzzzz', 'yessssss', 'yesss', 'gt', 'gtgtgt', 'etcwith', 'etchowever', 'etcwhy',
                        'etcthere', 'etcthey', 'etcpoc', 'zzzzzz', 'etcit', 'etcthe', 'etcon', 'etcl', 'pls', 'got',
                        'aint', 'arent', 'cuz', 'bc', 'aka', 'says', 'zvese', 'zvenyu', 'exp', 'zvefeja', 'exodus3414',
                        'zuuuuune', 'zuniga', 'zulum', '100']

s_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer_custom = CountVectorizer(max_df = 0.60, min_df = 0.05, stop_words = s_words)

counterberto2 = BERTopic(language='english', 
                        calculate_probabilities=True, 
                        verbose=True, 
                        #hdbscan_model = hdbscan_custom3,
                        vectorizer_model = vectorizer_custom)

topics, probabilities = counterberto2.fit_transform(counterhate_corpus, counterhate_embedding_array)

2021-07-29 15:53:43,009 - BERTopic - Reduced dimensionality with UMAP
2021-07-29 15:53:56,414 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [92]:
counterberto2.get_topic_info()

,Topic,Count,Name
0,-1,7334,-1_race_problem_ive_news
1,0,387,0_kung_spanish_started_times
2,1,298,1_jesus_amen_pray_allah
3,2,275,2_novel_moron_xi_fool
4,3,216,3_rate_symptoms_death_likely
...,...,...,...
152,155,11,155_false_zero_focus_folks
150,157,11,157_shouldnt_republicans_held_justify
149,153,11,153_international_wake_means_caused
159,158,10,158_happy_friend_flu_focus


In [93]:
counterberto2.visualize_topics()

In [94]:
counterberto2.visualize_barchart(top_n_topics=6, n_words = 10)

### Hate Tweets

Here, we inputed custom parameters for HDBSCAN and the count vectorizer. 

Clustering parameters: Clusters must have 25 members or more. Clusters must be more than 1.0 Euclidean units away from each other.

Vectorizer parameters: Words must appear between (0.05, 0.70) of the documents in the corpus (or else they will be filtered out as stop words).

In [49]:
hate_tweets = hydrated_data1.loc[hydrated_data1.Label == 'Hate']

#ensure that the dimensions are the same
ids = hate_tweets['tweet_id'].tolist()
hate_embeddings = embeddings[embeddings[0].isin(ids)]

id2 = hate_embeddings[0].tolist()
hate_tweets2 = hate_tweets[hate_tweets['tweet_id'].isin(id2)]
hate_tweets2 = hate_tweets2.set_index('tweet_id')
hate_tweets2 = hate_tweets2.loc[id2]

hate_embeddings = hate_embeddings.iloc[: , 1:]
hate_embedding_array = hate_embeddings.to_numpy()

In [50]:
random_state = 12000

additional_stop_words = ["ur", '21', 'amb', 'blah', 'till', 'yep', 'rt', 'yall', '23', '19', 'guy', 'hes', '14',
                        'okay', 'whats', 'huh', 'jan', 'did', 'didnt', 'mean', 'does', 'youre', 'yeah', 'hey', 'just',
                        'like', 'yes', 'yessss', 'yesssss', 'plz', '40', 'agree', 'zy', 'zvinopfuura', 'yess',
                         'zzzzzzzz', 'zzzzzzzz', 'yessssss', 'yesss', 'gt', 'gtgtgt', 'etcwith', 'etchowever', 'etcwhy',
                        'etcthere', 'etcthey', 'etcpoc', 'zzzzzz', 'etcit', 'etcthe', 'etcon', 'etcl', 'pls', 'got',
                        'aint', 'arent', 'cuz', 'bc', 'aka', 'says', 'zvese', 'zvenyu', 'exp', 'zvefeja', 'exodus3414',
                        'zuuuuune', 'zuniga', 'zulum', '100']

s_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)
vectorizer_custom = CountVectorizer(max_df = 0.70, min_df = 0.05, stop_words = s_words)

hate_corpus = hate_tweets2.apply(lambda x: cleaner2(x['text']), axis=1)
hateberto = BERTopic(language='english', 
                     calculate_probabilities=True, 
                     verbose=True, 
                     hdbscan_model = hdbscan_custom2,
                     vectorizer_model = vectorizer_custom)
topics, probabilities = hateberto.fit_transform(hate_corpus, hate_embedding_array)

2021-07-29 02:59:54,118 - BERTopic - Reduced dimensionality with UMAP
2021-07-29 03:00:05,122 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [51]:
hateberto.get_topic_info()

,Topic,Count,Name
0,-1,50930,-1_world_chinese_wuhan_coronavirus
1,0,2863,0_chinese_virus_people_world
2,1,498,1_ccp_end_english_coronavirus
3,2,73,2_abusing_vets_cat_necessarily
4,3,55,3_fuck_designer_bitlord_ir
5,4,55,4_die_gonna_going_fucking
6,5,47,5_kung_everybody_fighting_flu
7,6,43,6_tsla_spy_qqq_aapl
8,7,42,7_tong_ccpvirus_ting_thaitai
9,8,38,8_lennon_sean_rips_experimenter


In [52]:
hateberto.visualize_topics()

In [53]:
hateberto.visualize_barchart(top_n_topics=6, n_words = 10)

Topic 0 and topic 1 both feature ccp and communist rhetoric. Topic 2 seems to be about the banning of markets. Topic 3 features hateful rhetoric.

In [2]:
def read_embeddings(path):
    '''
    Function to read embeddings as dataframe and create a numpy array with the embeddings
    parameters: path - path for pickle file
    output - df_embeddings - dataframe, embeddings - numpy array
    '''
    df_embeddings = pd.read_pickle(path)
    embeddings = []
    for paper_id, row in df_embeddings.iterrows():
      embeddings.append(row['embedding'])
    embeddings = np.array(embeddings)
    return df_embeddings, embeddings

def plt_usingUMAP(data):
    '''
    Function to visualize high dimensional data by using UMAP dimensionality reduction
    parameters: data - High dimensional data as nparray
    output: plt - matplotlib plot object, standard_embedding - high dimensionality data reduced to 2D
    '''
    standard_embedding = umap.UMAP(random_state=42,n_components=2).fit_transform(data)
    plt.figure(figsize=(20, 15))
    plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1])
    return plt, standard_embedding


def clustering(embeddings,standard_embedding,n_components=100,min_samples=25,min_cluster_size=75):
    '''
    Function to reduce dimensions using UMAP and cluster the reduced data
    parameters: embeddings - high dimensional data, standard_embedding - data with reduced dimensions to 2D
    n_components - required dimensions after applying UMAP, min_samples - hdbscan input, min_cluster_size - hdbscan input
    output: clusterer - hdbscan object, labels - labels generated by hdbscan, plt - matplotlib plot object
    
    '''
    clusterable_embedding = umap.UMAP(
        n_neighbors=30,
        min_dist=0.0,
        n_components=n_components,
        random_state=42,
    ).fit_transform(embeddings)
    
    clusterer = hdbscan.HDBSCAN(
    min_samples=min_samples,
    min_cluster_size=min_cluster_size).fit(clusterable_embedding)
    
    labels = clusterer.labels_
    
    clustered = (labels >= 0)
    plt.figure(figsize=(20, 15))
    plt.scatter(standard_embedding[~clustered, 0],
                standard_embedding[~clustered, 1],
                c=(0.5, 0.5, 0.5),
                s=0.1,
                alpha=0.5)
    plt.scatter(standard_embedding[clustered, 0],
                standard_embedding[clustered, 1],
                c=labels[clustered],
                s=0.1,
                cmap='Spectral');
    return clusterer, labels, plt